In [1]:
import sys

sys.path.append('../..')

# Step of the underlying

In [2]:
step = 1/360.

# Max maturity of underlying

In [3]:
maturity = 5.

# Data

In [4]:
N = 20000

## Underlyings

In [5]:
from maths.montecarlo.processes.brownianmotions import GeometricBrownianMotion

time_grid = GeometricBrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
drift = 0.02
vol = 0.3

underlyings = [GeometricBrownianMotion(time_grid, x_0, drift, vol) for _ in range(N)]

## Randomization variables

In [6]:
from scipy.stats import expon
 
mean = maturity / 2.
exp_distrib = expon(loc=0, scale=mean)

print "Mean = %s"%mean

Mean = 2.5


In [7]:
from utils import v_time_offseter

In [8]:
zetas = exp_distrib.rvs(size=N)

zetas = v_time_offseter(zetas, time_grid)

print zetas.shape

(20000L,)


## Load intensities of obligors

In [9]:
import dill

raw_intensities = None
with open('../intensities/intensities_no_I5_new.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

['groups', 'idio']

In [10]:
import numpy as np

raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [11]:
subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(d_idio['hazard_rates'])
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(d_groups['hazard_rates'])
    pillars.append(d_groups['pillars'])

## Copula

In [12]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)

In [13]:
# We do not take into account the
# senior tranch of the CDO which
# has not been calibrated

bc_ids = list(raw_groups[-1]['ids'])

bc_subsets_indexes = []
for id_ in bc_ids:
    bc_subsets_indexes += copula.get_indexes_including(id_)
    
bc_subsets_indexes = list(set(bc_subsets_indexes))
bc_subsets_indexes.sort()

print bc_subsets_indexes

[0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124, 125, 126, 127, 128]


## Default times

In [14]:
default_times_mat = copula.generate_default_times(subsets_indexes=bc_subsets_indexes, number=N)

In [15]:
default_times_mat_shifted = v_time_offseter(default_times_mat, time_grid)

# Saving

In [16]:
to_save = {}

to_save["N"] = N
to_save["randomization"] = {
    "distrib": {"loc": 0., "scale": mean},
    "zetas": zetas,
    "maturity": maturity,
}
to_save["credit"] = {
    "copula": copula,
    "bc_subsets_indexes": bc_subsets_indexes,
    "bc_ids": bc_ids,
    "default_times": default_times_mat_shifted,
}
to_save["underlyings"] = underlyings

In [17]:
import dill

with open('data.pkl', 'wb') as f:
    dill.dump(to_save, f)